In [ ]:
import requests
from lxml import etree
import json
from time import sleep

In [ ]:
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/80.0.3987.162 Safari/537.36 Edg/80.0.361.109" }

In [ ]:
def get_aff(url, headers, id, page):
    data = {
        "limit": 30,
        "timeout": 3000,
        "filterTags": [],
        "tagId": id,
        "fromLemma": "true",
        "contentLength": 38,
        "page": page }
    response = requests.post(url, headers=headers, data=data)
    jdata = json.loads(response.content)
    urls = [(i['lemmaTitle'], i['lemmaUrl']) for i in jdata['lemmaList']]
    total_page = jdata['totalPage'] # the range of crawling
    return (urls, total_page)

In [ ]:
id = 76604
total_page = get_aff('https://baike.baidu.com/wikitag/api/getlemmas', headers, id, 0)[1]
urls = []
for i in range(total_page):
    sleep(0.05) # Avoid crawling too fast
    new_urls = get_aff('https://baike.baidu.com/wikitag/api/getlemmas', headers, id, i)[0]
    for url in new_urls:
        # Filter out the duplicated pages
        if url[1] not in urls:
            urls.append(url)

In [ ]:
info = { }
for url in urls:
    sleep(0.05)
    response = requests.get(url[1], headers=headers)
    response.ecoding = 'urf-8'
    tree = etree.HTML(response.text)
    entry = url[0]
    summary = ''.join(tree.xpath('//div[@class="lemma-summary"]//text()')).replace('\n', '').replace('	', '')
    print(entry)
    
    if entry in info:
        n = 2
        while "{} ({})".format(entry, n) in info:
            n += 1
        entry = "{} ({})".format(entry, n)
    info[entry] = summary

In [ ]:
with open("baike_data.json", "w", encoding = "utf8") as f:
    json.dump(info, f, ensure_ascii=False)